In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#/kaggle/input/happy-whale-and-dolphin/sample_submission.csv
#/kaggle/input/happy-whale-and-dolphin/train.csv
#/kaggle/input/happy-whale-and-dolphin/train_images/80b5373b87942b.jpg

In [ ]:
# Load the train data

train_df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
train_df.head(5)

In [ ]:
# There are 51K images in train set
train_df.info()

In [ ]:
# Sample submission
sample_sub = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
sample_sub.head(5)

In [ ]:
#Size of train set
train_df.shape

In [ ]:
# # Load some images
# from tqdm.autonotebook import tqdm
# from keras.preprocessing import image
# from keras.applications.imagenet_utils import preprocess_input

# # Loading images 
# # ds = training dataframe dataset , s = size of train_df, d = 'path to train_df'

# def load_images(ds, s , d):
#     x_train = np.zeros((s, 32, 32, ds.shape[1]))
#     c = 0
#     for fig in tqdm(ds['image']):
#         img = image.load_img("../input/happy-whale-and-dolphin/"+
#                              d+"/"+fig, 
#                              target_size=(32, 32, 3))
#         x = image.img_to_array(img)
#         x = preprocess_input(x)
#         x_train[c] = x
#         c += 1
#     return x_train
# X = load_images(train_df, train_df.shape[0], 'train_images')
# X /= 255
                         

In [ ]:
train_df.head(5)

In [ ]:
# Load images in training data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   height_shift_range = 0.1,
                                   width_shift_range = 0.1,
                                validation_split = 0.2,
                                horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

train_dir = '/kaggle/input/happy-whale-and-dolphin/train_images/'

train_set = train_datagen.flow_from_dataframe(train_df, train_dir,
                                              seed = 101,
                                            target_size = (64,64),
                                            batch_size = 32,
                                            x_col='image',
                                            y_col='individual_id',
                                            class_mode = 'categorical',
                                            subset = 'training')

valid_set = valid_datagen.flow_from_dataframe(train_df, 
                                              train_dir,
                                                seed = 101,
                                                target_size = (64,64),
                                              x_col='image',
                                                y_col='individual_id',
                                                batch_size = 32,
                                                class_mode = 'categorical',
                                                subset = 'validation')

In [ ]:
# '''
# Encode individual id's present in training data using 
# one hot encoding and label encoding
# Writing a function to retrurn labels for individual image id's
# ie: train_df['individual_id']

# '''
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# def labels(l):
#     values = np.array(l)
#     le = LabelEncoder()
#     oe = OneHotEncoder(parse = False)
#     le_values = le.fit_transform(values)
#     oe_values = oe.fit_transform(values)
#     return le_values, oe_values

# y, le = labels(train_df['individual_id'])


In [ ]:
#ref: https://www.kaggle.com/ammarnassanalhajali/cnn-with-keras-stater

# Build the Model

In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras import Sequential

model = Sequential()

model.add(Conv2D(32, (6,6), strides = (1,1), input_shape =
                (64,64,3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), strides = (1,1)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.85))

model.add(Dense(15587, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience = 3,
                          monitor = 'accuracy',
                          mode = 'max')]

In [ ]:
hist = model.fit(train_set,
                 epochs = 10,
                 validation_data = valid_set,
                 batch_size = 40000,
                 verbose = 1,
                callbacks = callbacks)


# Model Evaluation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15,5))
plt.plot(hist.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:

plt.figure(figsize = (15,5))
plt.plot(hist.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
print(hist.history['accuracy'])
print(hist.history['val_accuracy'])
print(hist.history['loss'])

# Leave an Upvote👍 if you Like/Fork my work. It will help me Grow